# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
db_list = mongo.list_database_names()
if 'uk_food' in db_list:
    print("Database 'uk_food' exists.")
else:
    print("Database 'uk_food' not found.")

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 67d264c4d0c1856bfca3509d, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our new database
collections = db.list_collection_names()
print("Collections in 'uk_food' database:", collections)

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

In [ ]:
# Review a document in the establishments collection
sample_doc = establishments.find_one()
print("\nSample Document from 'establishments' collection:")
pprint(sample_doc)

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "New Test Restaurant",
    "LocalAuthorityName": "Canterbury",
    "RatingValue": "5",
    "scores": {"Hygiene": 3, "Structural": 4, "ConfidenceInManagement": 5},
    "geocode": {"latitude": "51.2802", "longitude": "1.0789"},  # Stored as string initially
    "BusinessType": "Restaurant/Cafe/Canteen"
}

In [ ]:
# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_restaurant)
print(f"New restaurant inserted with ID: {insert_result.inserted_id}")

In [ ]:
# Check that the new restaurant was inserted
inserted_doc = establishments.find_one({"_id": insert_result.inserted_id})
print("\nInserted Restaurant:")
pprint(inserted_doc)

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
)
print("\nBusinessTypeID for 'Restaurant/Cafe/Canteen':")
pprint(business_type)

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
if business_type:
    establishments.update_one(
        {"_id": insert_result.inserted_id},
        {"$set": {"BusinessTypeID": business_type["BusinessTypeID"]}}
    )
    print("\nUpdated new restaurant with BusinessTypeID.")

In [ ]:
# Confirm that the new restaurant was updated
updated_doc = establishments.find_one({"_id": insert_result.inserted_id})
print("\nUpdated Restaurant Document:")
pprint(updated_doc)

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"\nNumber of establishments in Dover: {dover_count}")

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})
print(f"\nNumber of documents deleted: {delete_result.deleted_count}")

In [ ]:
# Check if any remaining documents include Dover
remaining_dover = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"\nRemaining Dover establishments: {remaining_dover}")

In [ ]:
# Check that other documents remain with 'find_one'
print("\nSample document after deletions:")
pprint(establishments.find_one())

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many(
    {},
    [
        {"$set": {
            "geocode.latitude": {"$toDecimal": "$geocode.latitude"},
            "geocode.longitude": {"$toDecimal": "$geocode.longitude"}
        }}
    ]
)
print("\nUpdated longitude and latitude to Decimal.")

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

print("\nSet non 1-5 Rating Values to Null.")

In [ ]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {"RatingValue": {"$type": "string"}},
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)
print("\nUpdated RatingValue to Integer.")

In [ ]:
# Check that the coordinates and rating value are now numbers
sample_doc = establishments.find_one({}, {"geocode": 1, "RatingValue": 1, "_id": 0})
print("\nSample document after type conversions:")
pprint(sample_doc)